In [17]:
def read_sent(path):
    ents = []
    curEnts = []
    for line in open(path):
        line = line.strip()
        if line == '':
            ents.append(curEnts)
            curEnts = []
        elif line[0] == '#' and len(line.split('\t')) == 1:
            continue
        else:
            curEnts.append(line.split('\t')[1])
    return(ents)

def read_labels(path):
    ents = []
    curEnts = []
    for line in open(path):
        line = line.strip()
        if line == '':
            ents.append(curEnts)
            curEnts = []
        elif line[0] == '#' and len(line.split('\t')) == 1:
            continue
        else:
            curEnts.append(line.split('\t')[2])
    return(ents)

In [18]:
#returns list of lists
training_labels = read_labels("baseline_data/en_ewt-ud-train.iob2")
training_sent = read_sent("baseline_data/en_ewt-ud-train.iob2")

In [28]:
dev_labels = read_labels("baseline_data/en_ewt-ud-dev.iob2")
dev_sent = read_sent("baseline_data/en_ewt-ud-dev.iob2")

In [26]:
from transformers import AutoModelForMaskedLM,AutoTokenizer

tokzr = AutoTokenizer.from_pretrained('bert-base-cased')
model = AutoModelForMaskedLM.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,  # Adjust as needed
    per_device_train_batch_size=16,  # Adjust as needed
    logging_dir='./logs',
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=zip(training_sent, training_labels),
    eval_dataset= zip(dev_sent, dev_labels)
)

# Train the model
trainer.train()

NameError: name 'PartialState' is not defined

In [30]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_length):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        input_ids = encoding["input_ids"].flatten()
        attention_mask = encoding["attention_mask"].flatten()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": label  # Adjust if you have labels
        }

# Example usage:
train_dataset = CustomDataset(training_sent, training_labels, tokzr, 512)
eval_dataset = CustomDataset(dev_sent, dev_labels, tokzr, 512)


In [32]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,  # Adjust as needed
    per_device_train_batch_size=16,  # Adjust as needed
    logging_dir='./logs',
)

# Define trainer
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset= eval_dataset
)

# Train the model
trainer.train()

NameError: name 'PartialState' is not defined